In [1]:
import slam
from importlib import reload

xFormers not available
xFormers not available


In [2]:
model, matching = slam.get_models()

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [3]:
image_prefix = "indoor3"

In [4]:
slam.sample_video(image_prefix)

In [5]:
images = slam.get_images(image_prefix)
match_inputs = [slam.preprocess_image_matcher(x) for x in images]
depth_inputs = [slam.preprocess_image_depth(x) for x in images]
depths = [slam.get_depth(image=x, model=model) for x in depth_inputs]

In [10]:
reload(slam)
matches = slam.get_matches(inputs=match_inputs, model=matching, num_matches=-1)
matches_3d = slam.project_matches_to_3d(matches, depths, camera_params=slam.iPhone15ProSquareVideoCamera())

In [11]:
for i,m in enumerate(matches_3d):
    for x in m:
        if x.shape[0] == 0:
            print(i)

In [7]:
transforms = [slam.get_transform(m[0], m[1]) for m in matches_3d]
angles = [slam.decompose_rotation_matrix(t[0]) for t in transforms]
elevation = [a[2] for a in angles]
azimuth = [a[1] for a in angles]

for a, t in zip(angles, transforms):
    x,y,z = a
    print("{: <15}".format(f"x deg {x:.3f}") + "{: <15}".format(f"y deg {y:.3f}") + "{: <15}".format(f"z deg {z:.3f}"))

    x,y,z = t[1]
    print("{: <15}".format(f"x {x:.3f}") + "{: <15}".format(f"y {y:.3f}") + "{: <15}".format(f"z {z:.3f}"))
    print()

x deg -1.709   y deg 1.264    z deg 0.083    
x 0.048        y 0.094        z 0.094        

x deg 0.010    y deg 5.471    z deg -0.386   
x 0.105        y -0.075       z 0.020        

x deg 0.195    y deg 15.647   z deg -6.653   
x -0.032       y 0.039        z 0.249        

x deg -0.258   y deg 18.793   z deg -8.275   
x -0.053       y 0.051        z 0.041        

x deg 7.280    y deg 9.993    z deg -1.736   
x -0.037       y -0.234       z 0.054        

x deg -2.175   y deg -3.481   z deg 1.785    
x 0.027        y 0.263        z 0.408        

x deg -4.223   y deg -9.982   z deg 1.790    
x -0.004       y 0.322        z 0.308        

x deg -2.272   y deg -18.260  z deg 3.403    
x -0.209       y 0.168        z 0.480        

x deg -1.236   y deg -2.944   z deg 10.083   
x 0.698        y 0.153        z 0.317        

x deg -1.412   y deg -15.413  z deg 4.135    
x -0.105       y 0.071        z 0.140        

x deg -4.942   y deg -19.201  z deg 4.441    
x -0.303       y 0.279  

In [5]:
# idx = 1
# fig = slam.plot_matches(idx, images, matches)

In [6]:
# fig = create_pcd_fig(idx, depths, images)
# fig.show()

In [8]:
data = slam.training_preprocessor(
    image_prefix=image_prefix, 
    model=model,
    matching=matching,
    camera=slam.iPhone15ProBackCamera(),
    force_reload=False)


In [9]:
slam.visualize_camera_poses(data=data)